In [1]:
# conda install -c conda-forge selenium

In [11]:
# importing libraries
import pandas as pd
import numpy as np
import time as time

#selenium imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [12]:
driver = webdriver.Chrome()

In [13]:
driver.get("https://runsignup.com/Race/OH/Columbus/TBDBITL5K")

In [14]:
time.sleep(1)
elems = driver.find_elements(By.XPATH,"//button")
for elem in elems:
    # print(elem.get_attribute("innerText"))
    if "I Accept" in elem.get_attribute("innerText"):
        elem.click()
        break

In [15]:
time.sleep(1)
elems = driver.find_elements(By.XPATH,"//ul[@class='menu primaryMenu']/li")
for elem in elems:
    if "RESULTS" in elem.get_attribute("innerText"):
        elem.click()    
        break

time.sleep(1)
elems = elem.find_elements(By.XPATH,"./ul//*")
for elem in elems:
    if "RESULTS" in elem.get_attribute("innerText"):
        elem.click()    
        break

In [16]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

time.sleep(1)
elem = driver.find_element(By.XPATH,"//select[@id='resultsPerPageSelect']")
elem.click()

time.sleep(1)
for elem2 in elem.find_elements(By.XPATH,".//option"):
    if "All" in elem2.get_attribute("innerText"):
        elem2.click()

In [20]:
def get_results_df():

    table = driver.find_element(By.XPATH,"//table[@id='resultsTable']")
    tHeader = table.find_element(By.XPATH,"./thead")
    tBody = table.find_element(By.XPATH,"./tbody")

    columns = list()
    for elem in tHeader.find_elements(By.XPATH,"./tr/*"):
        col_name = (elem.get_attribute("innerText"))
        col_name = col_name.replace(" ","_").replace("\n","_")
        columns.append(col_name)

    df = pd.DataFrame(columns=columns)

    rowElems = tBody.find_elements(By.XPATH,"./tr")
    for rowElem in rowElems:
        temp_row = list()
        for cell in rowElem.find_elements(By.XPATH,"./*"):
            if len(cell.find_elements(By.XPATH,".//div")) == 0:
    #             print(cell.get_attribute("innerText"))
                temp_row.append(cell.get_attribute("innerText"))
            else:
                fNameElem = cell.find_element(By.XPATH,".//div[@class='participantName__name__firstName']")
                lNameElem = cell.find_element(By.XPATH,".//div[@class='participantName__name__lastName']")
                fNameStr = fNameElem.get_attribute("innerText")
                lNameStr = lNameElem.get_attribute("innerText")
    #             print(fNameStr + "_" + lNameStr)
                temp_row.append(fNameStr + "_" + lNameStr)

        temp_dict = {df.columns[i]: temp_row[i] for i in range(len(df.columns))}
        temp_df = pd.DataFrame(temp_dict,index=[0])

        df = pd.concat([df,temp_df])
        
    return df

In [21]:
get_results_df()

,Place,Bib,Name,Gender,Gender_Place,Age,City,State,Clock_Time,Chip_Time,Pace,Division_Place,Division
0,1,365,Aidan_Bowman,M,1,18,Hebron,OH,18:44.55,18:43.21,6:02,1,M Top Overall
0,2,1688,Tony_DeRosa,M,2,22,Fort Thomas,KY,18:57.37,18:55.38,6:05,,
0,3,985,Owen_Hennessey,M,3,21,Columbus,OH,19:05.38,19:03.13,6:08,,
0,4,944,Ryan_Morgan,M,4,43,Columbus,OH,19:46.40,19:43.12,6:21,,
0,5,371,Bobby_Petty,M,5,18,New Albany,OH,20:29.55,20:26.92,6:35,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,110,336,Mike_Lockhart,M,61,58,Dayton,OH,49:50.07,49:37.32,15:58,,
0,111,73,Jeannie_Heffner,F,49,55,Hilliard,OH,56:14.69,55:50.40,17:58,,
0,112,55,Jeff_Heffner,M,62,60,Hilliard,OH,56:15.34,55:50.54,17:58,,
0,113,961,Kimberly_Madigan,F,50,53,plain city,OH,56:48.79,56:18.39,18:07,,


In [10]:
driver.quit()